In [1]:
import time
import matplotlib.pyplot as plt
import torch
import numpy as np
from laplacian_py import laplacian_solver

resx = 80
resy = 30
resz = 50

resx = 2*64//1
resy = 2*64//1
resz = 2*64//1

max_val = 100/1
cost_scale = .1

values = .99*max_val*torch.ones((resx, resy, resz),dtype=torch.float32, device='cuda')
cost = cost_scale*torch.ones((resx,resy, resz),dtype=torch.float32, device='cuda')
boundary_conditions = torch.zeros((resx,resy, resz),dtype=torch.float32, device='cuda')
boundary_types = torch.zeros((resx,resy, resz),dtype=torch.float32, device='cuda')
# goal
goal_ind1 = int(resx*15/30)
goal_ind2 = int(resy*15/30)
# goal_ind3 = int(resz*27/30)
goal_ind3 = int(resz*15/30)
boundary_conditions[goal_ind1, goal_ind2, goal_ind3] = 0.0
boundary_types[goal_ind1, goal_ind2, goal_ind3] = 1.0


In [2]:
import sys
sys.path.append('../')

from scripts.point_cloud_utils import send_point_cloud
from pyngp.common import *
from tqdm import tqdm
import pyngp.pyngp as ngp
testbed = ngp.Testbed()
testbed.load_snapshot('../nerf/scene.ingp')

box_diff = testbed.aabb.max - testbed.aabb.min
min_dim = -box_diff / 2
max_dim = box_diff / 2

x_segments = np.linspace(.2, .8, resx)
y_segments = np.linspace(.45-.02, .59-.02, resy)
z_segments = np.linspace(.35, .65, resz)
x, y, z = np.meshgrid(x_segments, y_segments, z_segments, indexing='ij')
points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
points = points[0:256 * (points.shape[0] // 256), :]

dirs = np.zeros(points.shape)
dirs[:, 0] = 1
dirs[:, 1] = .5
dirs[:, 2] = .5

dt = np.zeros((points.shape[0], 1))
coords = np.hstack((points, dt, dirs))
nerf_values = testbed.sample_nerf(list(coords.flatten()))
nerf_values = np.reshape(nerf_values, (coords.shape[0], -1))

points = np.vstack((points[:, 0], points[:, 2], 1 - points[:, 1])).T
points = points * (max_dim - min_dim) + min_dim
points = 2 * points

send_point_cloud(np.hstack((points, nerf_values)), has_alpha=False, wait_time=5)

17:13:34 SUCCESS  Initialized CUDA 12.0. Active GPU is #0: NVIDIA GeForce RTX 4090 [89]
17:13:34 INFO     Loading network snapshot from: ../nerf/scene.ingp
17:13:36 DEBUG    GPUMemory: allocating 1.41 KB.
17:13:36 DEBUG    GPUMemory: allocating 144 B.
17:13:36 DEBUG    GPUMemory: allocating 144 B.
17:13:36 DEBUG    GPUMemory: allocating 144 B.
17:13:36 DEBUG    GPUMemory: allocating 144 B.
17:13:36 DEBUG    GPUMemory: allocating 8 B.
17:13:36 DEBUG    GPUMemory: allocating 1.12 KB.
17:13:36 DEBUG    GPUMemory: allocating 4 MB.
17:13:36 DEBUG    GPUMemory: allocating 8 MB.
17:13:36 DEBUG    GPUMemory: allocating 2 MB.
17:13:36 DEBUG    GPUMemory: allocating 64 KB.
17:13:36 INFO     GridEncoding:  Nmin=16 b=2 F=4 T=2^22 L=8
17:13:36 DEBUG    GridEncoding at level 0: resolution=16 params_in_level=4096
17:13:36 DEBUG    GridEncoding at level 1: resolution=32 params_in_level=32768
17:13:36 DEBUG    GridEncoding at level 2: resolution=64 params_in_level=262144
17:13:36 DEBUG    GridEncoding 

In [3]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from laplacian_py.network import LaplaceNet, compute_loss, calculate_gradient, interpolate_prediction
    
network = LaplaceNet((resx, resy, resz), max_val, 1*cost_scale, 2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
network = network.to(device)

/home/paul/CLionProjects/thesis_nerf/venv/lib/python3.10/site-packages/tinycudann-1.7-py3.10-linux-x86_64.egg/tinycudann/modules.py:53: UserWarning: tinycudann was built for lower compute capability (86) than the system's (89). Performance may be suboptimal.
  warnings.warn(f"tinycudann was built for lower compute capability ({cc}) than the system's ({system_compute_capability}). Performance may be suboptimal.")


In [4]:
lr = 1e-4 # norm
momentum = 0.8
optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum)

XYZ_train = [np.linspace(-1, 1, res, dtype=np.float32) for res in [resx, resy, resz]]
X_train, Y_train, Z_train = np.meshgrid(*XYZ_train)
out = np.stack([X_train, Y_train, Z_train], axis=3)
grid_train = torch.tensor(out, dtype=torch.float32, device='cuda', requires_grad=True)
grid_train = grid_train.unsqueeze(axis=0)

target = torch.tensor(10+20*np.sqrt(X_train**2+Y_train**2+Z_train**2),dtype=torch.float32, device='cuda')
# target = torch.tensor(10+20*np.sqrt(X_train**2+Z_train**2),dtype=torch.float32, device='cuda')
# target = torch.tensor(21 + 20*Y_train,dtype=torch.float32, device='cuda')

    
network.reset()
for it in range(0, 200):
    optimizer.zero_grad()
    pred, C, _ = network(grid_train, boundary_types, boundary_conditions)
    loss_train = 1*compute_loss(pred=pred, target=target, C=C, cost_scale=cost_scale)
    if it % 10 == 0:
        print(loss_train)
    loss_train.backward()
    optimizer.step()
    
    
# network = torch.load('network_3d.pt')

/home/paul/CLionProjects/thesis_nerf/nerf_ws/install/laplacian_py/lib/python3.10/site-packages/laplacian_py/network.py:32: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  extra_cost = F.conv3d(objects_bounds.unsqueeze(dim=0), weight, bias=None, stride=1, padding='same',


tensor(5145.8896, device='cuda:0', grad_fn=<MulBackward0>)
tensor(451.2219, device='cuda:0', grad_fn=<MulBackward0>)
tensor(239.4558, device='cuda:0', grad_fn=<MulBackward0>)
tensor(205.1891, device='cuda:0', grad_fn=<MulBackward0>)
tensor(236.7660, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.2232, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8171, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8169, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8169, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8169, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8169, device='cuda:0', grad_fn=<MulBackward0>)
tensor(250.8168, device='cuda:0', grad_fn=<MulBackward0>)



KeyboardInterrupt



In [5]:
boundary_types_2 = boundary_types.clone()
boundary_conditions_2 = boundary_conditions.clone()
 
obj_inds = nerf_values[:,3] > 100/255
obj_inds = np.reshape(obj_inds, (resx, resy, resz))
obj_inds = torch.tensor(obj_inds) # this should not be needed
boundary_conditions_2[obj_inds] = max_val*10
boundary_types_2[obj_inds] = 1.0

# goal_ind1 = int(resx*15/30)
# goal_ind2 = int(resy*15/30)
# goal_ind3 = int(resz*15/30)
# boundary_conditions_2[goal_ind1, goal_ind2, goal_ind3] = 0.0
# boundary_types_2[goal_ind1, goal_ind2, goal_ind3] = 0.0

# goal_ind1 = int(resx*15/30)
# goal_ind2 = int(resy*15/30)
# goal_ind3 = int(resz*27/30)
# boundary_conditions_2[goal_ind1, goal_ind2, goal_ind3] = 0.0
# boundary_types_2[goal_ind1, goal_ind2, goal_ind3] = 1.0

# network.reset()
for i in range(1000):
    if i % 100 ==0:
        print(i)
    pred, C_pos, J2 = network(grid_train, boundary_types_2, boundary_conditions_2)
    V_in = pred.detach()

0
100
200
300
400
500
600
700
800
900


In [113]:
import rclpy
from rclpy.node import Node
from visualization_msgs.msg import MarkerArray, Marker
from geometry_msgs.msg import Point

if not rclpy.ok():
    rclpy.init()
node = Node('visualizer_node')
pub = node.create_publisher(MarkerArray, 'visualization_lines', 10)
    
msg = MarkerArray()

dt = 0.001
marker_id = 0
grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)

x_min = points[0,0]
y_min = points[0,1]
z_min = points[0,2]
x_max = points[-1,0]
y_max = points[-1,1]
z_max = points[-1,2]

# x_segments = np.linspace(.2, .8, resx)
# y_segments = np.linspace(.45-.02, .59-.02, resy)
# z_segments = np.linspace(.35, .65, resz)


for z_val in [-.95, -.5, 0, .5 ,.95]:
    for y_val in [-.95, -.5, 0, .5 ,.95]:
        for x_val in [-.95, 0.0, .95]:
            cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
            traj = np.zeros((2000, 3))
            for i in range(traj.shape[0]):
                query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
                query.unsqueeze(axis=0)
                J1, J2 = calculate_gradient(grid_pred, query)
                J2 = J2.cpu().detach().numpy()
#                 J2[abs(J2) != np.max(abs(J2))] = 0
                J2 = J2/(np.linalg.norm(J2) + .001)
                cur += -J2*dt
                traj[i, :] = cur.squeeze()
    
            marker = Marker()
            marker.header.frame_id = "unity";
            marker.id = marker_id;
            marker_id += 1
            marker.type = marker.LINE_STRIP;
            marker.action = marker.ADD;
            
            for ind in range(traj.shape[0]):
                position = Point()
                position.x = .5*(traj[ind, 2]+1.0)*(x_max-x_min) + x_min
                position.y = .5*(traj[ind, 1]+1.0)*(y_max-y_min) + y_min
                position.z = .5*(traj[ind, 0]+1.0)*(z_max-z_min) + z_min
                marker.points.append(position)

            marker.scale.x = .0025;
            marker.scale.y = 0.0;
            marker.scale.z = 0.0;
            marker.color.a = 1.0; 
            marker.color.r = 0.0;
            marker.color.g = 0.0;
            marker.color.b = 1.0;

            msg.markers.append(marker)


pub.publish(msg)


KeyboardInterrupt: 

In [7]:
for i in range(100):
    pub.publish(msg)

In [115]:
import rclpy
from rclpy.node import Node
from tf2_ros.transform_listener import TransformListener
from tf2_ros.buffer import Buffer
from tf2_ros import TransformException
from std_msgs.msg import Float32MultiArray
import sys
sys.path.append('../')
from scripts.ros_utils import ros_tf_to_matrix



if not rclpy.ok():
    rclpy.init()
node = Node('run_network')
    
vel_pub = node.create_publisher(Float32MultiArray, '/operational_velocity', 10) 
tf_buffer = Buffer()
tf_listener = TransformListener(tf_buffer, node)


def get_transform_ee(base_frame, frame):
    msg = None
    while msg == None:
            rclpy.spin_once(node)
            try:
                msg = tf_buffer.lookup_transform(base_frame, frame,
                                                 rclpy.time.Time())
            except TransformException as ex:
                pass

    return ros_tf_to_matrix(msg)


pos = get_transform_ee('base', 'right_gripper_l_finger_tip')[:3, 3]
goal = pos.copy()
goal[2] += .00
goal[1] += 0.1
goal[0] += 0.0
while np.linalg.norm(pos - goal) > .01:
    pos = get_transform_ee('base', 'right_gripper_l_finger_tip')[:3, 3]
    xd = np.zeros(6,)
    xd[:3] = 1*(goal - pos)
    vel_msg = Float32MultiArray()
    vel_msg.data = list(xd)
    vel_pub.publish(vel_msg)
 




[WARN] [1698962681.103765201] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


KeyboardInterrupt: 

In [ ]:
       

pos_unity = get_transform_ee('unity', 'right_gripper_l_finger_tip')[:3, 3]
Tbase_unity = get_transform_ee('base', 'unity')

dt = 0.005
grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)
while True:
    pos_unity = get_transform_ee('unity', 'right_gripper_l_finger_tip')[:3, 3]
    cur = np.reshape(np.array([pos_unity[0], pos_unity[1], pos_unity[2]]),(1, 1, 1, 1, 3))
    query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
    query.unsqueeze(axis=0)
    J1, J2 = calculate_gradient(grid_pred, query)
    J2 = J2.cpu().detach().numpy()
    J2 = J2/(np.linalg.norm(J2) + .001)
    J2 = J2.squeeze()
#     J2x = J2[0]
#     J2z = J2[2]
#     J2[0] = J2z
#     J2[2] = J2x
    
    if np.any(np.isnan(J2)):
        print("nan!")
        J2 = .1*pos_unity/np.linalg.norm(pos_unity)
        
    goal_unity = pos_unity - J2*dt    
    print('new goal')
    goal =  (Tbase_unity[:3,:3] @ goal_unity) + Tbase_unity[:3,3]
    it = 0
    while  it < 10:
        pos = get_transform_ee('base', 'right_gripper_l_finger_tip')[:3, 3]
        xd = np.zeros(6,)
        xd[:3] = 3*(goal - pos)
        vel_msg = Float32MultiArray()
        vel_msg.data = list(xd)
        vel_pub.publish(vel_msg)
        it+=1


new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal
new goal


In [29]:
pos_unity

array([0.01530059, 0.00806129, 0.00814947])

In [30]:
goal_unity

array([ 0.01528889,  0.00805469, -0.00185052])

In [18]:
Tbase_unity

array([[ 2.22044605e-16,  1.00000000e+00,  0.00000000e+00,
         6.58949256e-01],
       [-1.00000000e+00,  2.22044605e-16, -0.00000000e+00,
        -7.35132322e-02],
       [-0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         5.47477603e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [19]:
pos

array([ 0.85801135, -0.04897862,  0.10692175])

In [20]:
xd

array([ 0.00000000e+00,  3.12250226e-17, -4.16333634e-18,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [76]:
goal_unity-pos_unity

array([-0.02719753,  0.09231465,  0.0271705 ])

In [ ]:
# J1, J2 = calculate_gradient(grid_pred, query)
# J2

In [78]:
J2

array([[[[[ 0.27197534, -0.9231464 , -0.27170497]]]]], dtype=float32)